In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hlh
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
import_lustre_dir="file:/lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge"
sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/hail-20211108-1311-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [49]:
mt1=hl.read_matrix_table(f"{lustre_dir}/ukbb_hits.mt")
mt2=hl.read_matrix_table(f"{lustre_dir}/ibd_hits.mt")
mt=hl.read_matrix_table(f"{lustre_dir}/intersection_hits.mt")

In [51]:
mt1=hl.split_multi_hts(mt1)
mt2=hl.split_multi_hts(mt2)
mt1.write(f"{lustre_dir}/ukbb_hits_split.mt")
mt2.write(f"{lustre_dir}/ibd_hits_split.mt")

2021-11-08 11:02:28 Hail: INFO: Coerced sorted dataset
2021-11-08 11:02:41 Hail: INFO: wrote matrix table with 61 rows and 200643 columns in 20 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_hits_split.mt
    Total size: 38.32 MiB
    * Rows/entries: 37.19 MiB
    * Columns: 1.13 MiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (372.88 KiB)
    * Largest partition:  26 rows (16.90 MiB)
2021-11-08 11:02:44 Hail: INFO: wrote matrix table with 50 rows and 21465 columns in 1 partition to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ibd_hits_split.mt
    Total size: 3.73 MiB
    * Rows/entries: 3.65 MiB
    * Columns: 84.29 KiB
    * Globals: 11.00 B
    * Smallest partition: 50 rows (3.65 MiB)
    * Largest partition:  50 rows (3.65 MiB)


In [52]:
mt1=hl.read_matrix_table(f"{lustre_dir}/ukbb_hits_split.mt")
mt2=hl.read_matrix_table(f"{lustre_dir}/ibd_hits_split.mt")

In [53]:
#UKBB 
mt1.count()

(61, 200643)

In [54]:
#IBD
mt2.count()

(50, 21465)

In [55]:
mt_intersection=mt1.union_cols(mt2)

In [56]:
mt_intersection.write(f"{lustre_dir}/intersection_hits_split.mt")

2021-11-08 11:12:37 Hail: INFO: wrote matrix table with 48 rows and 222108 columns in 20 partitions to file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/intersection_hits_split.mt
    Total size: 34.96 MiB
    * Rows/entries: 33.75 MiB
    * Columns: 1.21 MiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (424.15 KiB)
    * Largest partition:  21 rows (15.19 MiB)


In [5]:
#INTERSECTION COUNT  is now 48 (up from 34)
mt_intersection=hl.read_matrix_table(f"{lustre_dir}/intersection_hits_split.mt")
mt_intersection.count()

(48, 222108)

In [ ]:
mt_intersection

In [59]:
mt1.rows().export(f"{lustre_dir}/ukbb_hits_variants_split.tsv.bgz" , delimiter="\t")
mt2.rows().export(f"{lustre_dir}/ibd_hits_variants_split.tsv.bgz" , delimiter="\t")
mt_intersection.rows().export(f"{lustre_dir}/intersection_hits_variants_split.tsv.bgz" , delimiter="\t")



2021-11-08 11:18:32 Hail: INFO: merging 20 files totalling 3.9K...
2021-11-08 11:18:33 Hail: INFO: while writing:
    file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_hits_variants_split.tsv.bgz
  merge time: 257.320ms
2021-11-08 11:18:33 Hail: INFO: merging 1 files totalling 1.3K...
2021-11-08 11:18:33 Hail: INFO: while writing:
    file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ibd_hits_variants_split.tsv.bgz
  merge time: 24.081ms
2021-11-08 11:18:33 Hail: INFO: merging 20 files totalling 3.7K...
2021-11-08 11:18:34 Hail: INFO: while writing:
    file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/intersection_hits_variants_split.tsv.bgz
  merge time: 259.739ms


In [60]:
ukb_hits=hl.import_table(f"{lustre_dir}/ukbb_hits_variants_split.tsv.bgz")
ibd_hits=hl.import_table(f"{lustre_dir}/ibd_hits_variants_split.tsv.bgz")
intesection_hits=hl.import_table(f"{lustre_dir}/intersection_hits_variants_split.tsv.bgz")

2021-11-08 11:19:15 Hail: INFO: Reading table without type imputation
  Loading field 'locus' as type str (not specified)
  Loading field 'alleles' as type str (not specified)
  Loading field 'rsid' as type str (not specified)
  Loading field 'qual' as type str (not specified)
  Loading field 'filters' as type str (not specified)
  Loading field 'info' as type str (not specified)
  Loading field 'a_index' as type str (not specified)
  Loading field 'was_split' as type str (not specified)
2021-11-08 11:19:15 Hail: INFO: Reading table without type imputation
  Loading field 'locus' as type str (not specified)
  Loading field 'alleles' as type str (not specified)
  Loading field 'rsid' as type str (not specified)
  Loading field 'qual' as type str (not specified)
  Loading field 'filters' as type str (not specified)
  Loading field 'info' as type str (not specified)
  Loading field 'a_index' as type str (not specified)
  Loading field 'was_split' as type str (not specified)
2021-11-08 11:

In [61]:
ukb_hits_pd=ukb_hits.to_pandas()
ibd_hits_pd=ibd_hits.to_pandas()
merged_pd_split=pd.merge(ukb_hits_pd, ibd_hits_pd, how='outer', on=['locus'], indicator="In both or unique", suffixes=('_ukbb', '_ibd' ))
merged_pd_split.to_csv("/home/ubuntu/data/scripts/merged_split_mt.tsv", sep="\t",index=False)

In [11]:
bed=hl.import_bed("file:///lustre/scratch123/projects/wes_jc_ukb_ibd/glnexus_ibd/ibd_ukb_intersect_capture_not_padded.bed")
bed.show()

2021-11-08 13:18:03 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)


""
interval
interval<locus<GRCh38>>
[chr1:69482-chr1:69601)
[chr1:925942-chr1:926014)
[chr1:930203-chr1:930337)
[chr1:931039-chr1:931090)
[chr1:935772-chr1:935883)
[chr1:939040-chr1:939130)
[chr1:939272-chr1:939437)
[chr1:941144-chr1:941307)


In [13]:
mt1=hl.read_matrix_table(f"{lustre_dir}/ukbb_hits_split.mt")
mt2=hl.read_matrix_table(f"{lustre_dir}/ibd_hits_split.mt")

mt_intersection=hl.read_matrix_table(f"{lustre_dir}/intersection_hits_split.mt")

print(mt1.count())
print(mt2.count())
print(mt_intersection.count())
filtered_mt1 = mt1.filter_rows(hl.is_defined(bed[mt1.locus]))
filtered_mt2 = mt2.filter_rows(hl.is_defined(bed[mt2.locus]))
filtered_mt_int = mt_intersection.filter_rows(hl.is_defined(bed[mt_intersection.locus]))
print(filtered_mt1.count())
print(filtered_mt2.count())
print(filtered_mt_int.count())

(61, 200643)
(50, 21465)
(48, 222108)


2021-11-08 13:22:51 Hail: INFO: Ordering unsorted dataset with network shuffle


(49, 200643)


2021-11-08 13:22:59 Hail: INFO: Ordering unsorted dataset with network shuffle


(41, 21465)


2021-11-08 13:23:08 Hail: INFO: Ordering unsorted dataset with network shuffle


(39, 222108)


In [20]:
variants_not_in_capture_region = mt_intersection.filter_rows(hl.is_defined(bed[mt_intersection.locus])==False)

variants_not_in_capture_region.count()

2021-11-08 13:25:52 Hail: INFO: Ordering unsorted dataset with network shuffle


(9, 222108)

In [22]:
variants_not_in_capture_region.show()

2021-11-08 13:26:28 Hail: INFO: Ordering unsorted dataset with network shuffle


,
locus,alleles
locus<GRCh38>,array<str>
chr2:233527423,"[""C"",""T""]"
chr4:94573345,"[""C"",""T""]"
chr9:136365140,"[""C"",""G""]"
chr9:136464447,"[""C"",""T""]"
chr12:40346884,"[""A"",""G""]"
chr16:50699481,"[""T"",""A""]"
chr16:50716671,"[""G"",""A""]"
chr16:50716671,"[""G"",""GT""]"


In [ ]:
ukb_hits_split=hl.import_table(f"{lustre_dir}/ukbb_hits_variants_split.tsv.bgz")
ibd_hits_split=hl.import_table(f"{lustre_dir}/ibd_hits_variants.tsv.bgz")
intesection_hits_split=hl.import_table(f"{lustre_dir}/intersection_hits_variants.tsv.bgz")

In [9]:
ukb_hits=hl.import_table(f"{lustre_dir}/ukbb_hits_variants.tsv.bgz")
ibd_hits=hl.import_table(f"{lustre_dir}/ibd_hits_variants_split.tsv.bgz")
intesection_hits=hl.import_table(f"{lustre_dir}/intersection_hits_variants.tsv.bgz")

2021-11-08 09:55:26 Hail: INFO: Reading table without type imputation
  Loading field 'locus' as type str (not specified)
  Loading field 'alleles' as type str (not specified)
  Loading field 'rsid' as type str (not specified)
  Loading field 'qual' as type str (not specified)
  Loading field 'filters' as type str (not specified)
  Loading field 'info' as type str (not specified)
2021-11-08 09:55:26 Hail: INFO: Reading table without type imputation
  Loading field 'locus' as type str (not specified)
  Loading field 'alleles' as type str (not specified)
  Loading field 'rsid' as type str (not specified)
  Loading field 'qual' as type str (not specified)
  Loading field 'filters' as type str (not specified)
  Loading field 'info' as type str (not specified)
2021-11-08 09:55:26 Hail: INFO: Reading table without type imputation
  Loading field 'locus' as type str (not specified)
  Loading field 'alleles' as type str (not specified)
  Loading field 'rsid' as type str (not specified)
  Loadi

In [10]:
ukb_hits.show()

,,,,,
locus,alleles,rsid,qual,filters,info
str,str,str,str,str,str
"""chr1:28150681""","""[""T"",""C""]""","""chr1_28150681_T_C""","""4.7000e+01""",NA,"""{""AF"":[0.002562],""AQ"":[47],""AC"":null,""AN"":null}"""
"""chr1:67206980""","""[""T"",""C""]""","""chr1_67206980_T_C""","""5.3000e+01""",NA,"""{""AF"":[0.007449],""AQ"":[53],""AC"":null,""AN"":null}"""
"""chr1:67240217""","""[""G"",""A"",""T""]""","""chr1_67240217_G_A;chr1_67240217_G_T""","""5.6000e+01""",NA,"""{""AF"":[0.01314,2.0E-6],""AQ"":[56,36],""AC"":null,""AN"":null}"""
"""chr1:67240275""","""[""G"",""A""]""","""chr1_67240275_G_A""","""5.3000e+01""",NA,"""{""AF"":[0.063558],""AQ"":[53],""AC"":null,""AN"":null}"""
"""chr2:233263166""","""[""A"",""G""]""","""chr2_233263166_A_G""","""5.2000e+01""",NA,"""{""AF"":[0.051622],""AQ"":[52],""AC"":null,""AN"":null}"""
"""chr2:233320674""","""[""A"",""G""]""","""chr2_233320674_A_G""","""5.9000e+01""",NA,"""{""AF"":[0.052015],""AQ"":[59],""AC"":null,""AN"":null}"""
"""chr2:233527418""","""[""ACTGCC"",""A""]""","""chr2_233527418_ACTGCC_A""","""5.1000e+01""","""[""MONOALLELIC""]""","""{""AF"":[2.0E-6],""AQ"":[51],""AC"":null,""AN"":null}"""
"""chr2:233527423""","""[""C"",""T""]""","""chr2_233527423_C_T""","""5.8000e+01""",NA,"""{""AF"":[0.046869],""AQ"":[58],""AC"":null,""AN"":null}"""


In [11]:
ibd_hits.show()

,,,,,
locus,alleles,rsid,qual,filters,info
str,str,str,str,str,str
"""chr1:28150681""","""[""T"",""C""]""","""chr1_28150681_T_C""","""5.1000e+01""",NA,"""{""AF"":[0.003541],""AQ"":[51],""AC"":null,""AN"":null}"""
"""chr1:67206980""","""[""T"",""C""]""","""chr1_67206980_T_C""","""5.5000e+01""",NA,"""{""AF"":[0.008526],""AQ"":[55],""AC"":null,""AN"":null}"""
"""chr1:67240217""","""[""G"",""A""]""","""chr1_67240217_G_A""","""5.9000e+01""",NA,"""{""AF"":[0.009294],""AQ"":[59],""AC"":null,""AN"":null}"""
"""chr1:67240275""","""[""G"",""A"",""T""]""","""chr1_67240275_G_A;chr1_67240275_G_T""","""5.7000e+01""",NA,"""{""AF"":[0.032075,1.4E-4],""AQ"":[57,32],""AC"":null,""AN"":null}"""
"""chr2:233263166""","""[""A"",""G""]""","""chr2_233263166_A_G""","""5.6000e+01""",NA,"""{""AF"":[0.05488],""AQ"":[56],""AC"":null,""AN"":null}"""
"""chr2:233320674""","""[""A"",""G""]""","""chr2_233320674_A_G""","""6.4000e+01""",NA,"""{""AF"":[0.049266],""AQ"":[64],""AC"":null,""AN"":null}"""
"""chr2:233527423""","""[""C"",""T""]""","""chr2_233527423_C_T""","""6.1000e+01""",NA,"""{""AF"":[0.043396],""AQ"":[61],""AC"":null,""AN"":null}"""
"""chr4:3447925""","""[""G"",""A""]""","""chr4_3447925_G_A""","""5.8000e+01""",NA,"""{""AF"":[0.07652],""AQ"":[58],""AC"":null,""AN"":null}"""


In [13]:
ukb_hits_locus=ukb_hits.key_by('locus')
ibd_hits_locus=ibd_hits.key_by('locus')

In [19]:
ukb_hits.count()

47

In [20]:
ibd_hits.count()

46

In [22]:
left=ukb_hits_locus.key_by('locus').join(ibd_hits_locus.key_by('locus'), 'left')

2021-11-08 10:07:27 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'filters' -> 'filters_1'
    'info' -> 'info_1'
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'
    'rsid' -> 'rsid_1'
    'qual' -> 'qual_1'


In [23]:
left.count()

2021-11-08 10:07:30 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-11-08 10:07:30 Hail: INFO: Ordering unsorted dataset with network shuffle


47

In [24]:
left.export(f"{lustre_dir}/ukbb_hits_variants_left_join_ibd.tsv.bgz")

2021-11-08 10:07:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-11-08 10:07:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-11-08 10:07:37 Hail: INFO: merging 1 files totalling 1.8K...
2021-11-08 10:07:37 Hail: INFO: while writing:
    file:///lustre/scratch123/mdt1/projects/wes_jc_ukb_ibd/hail_merge/ukbb_hits_variants_left_join_ibd.tsv.bgz
  merge time: 30.238ms


In [33]:
ukb_hits_pd=ukb_hits.to_pandas()
ibd_hits_pd=ibd_hits.to_pandas()

In [37]:
ukb_hits_pd.columns

Index(['locus', 'alleles', 'rsid', 'qual', 'filters', 'info'], dtype='object')

In [44]:
merged_pd=pd.merge(ukb_hits_pd, ibd_hits_pd, how='outer', on=['locus'], indicator="In both or unique", suffixes=('_ukbb', '_ibd' ))

In [45]:
merged_pd.head()

,locus,alleles,rsid_ukbb,qual_ukbb,filters_ukbb,info_ukbb,rsid_ibd,qual_ibd,filters_ibd,info_ibd,In both or unique
0,chr1:28150681,"[""T"",""C""]",chr1_28150681_T_C,4.7000e+01,None,"{""AF"":[0.002562],""AQ"":[47],""AC"":null,""AN"":null}",chr1_28150681_T_C,5.1000e+01,None,"{""AF"":[0.003541],""AQ"":[51],""AC"":null,""AN"":null}",both
1,chr1:67206980,"[""T"",""C""]",chr1_67206980_T_C,5.3000e+01,None,"{""AF"":[0.007449],""AQ"":[53],""AC"":null,""AN"":null}",chr1_67206980_T_C,5.5000e+01,None,"{""AF"":[0.008526],""AQ"":[55],""AC"":null,""AN"":null}",both
2,chr1:67240217,"[""G"",""A"",""T""]",chr1_67240217_G_A;chr1_67240217_G_T,5.6000e+01,None,"{""AF"":[0.01314,2.0E-6],""AQ"":[56,36],""AC"":null,...",NaN,NaN,NaN,NaN,left_only
3,chr1:67240275,"[""G"",""A""]",chr1_67240275_G_A,5.3000e+01,None,"{""AF"":[0.063558],""AQ"":[53],""AC"":null,""AN"":null}",NaN,NaN,NaN,NaN,left_only
4,chr2:233263166,"[""A"",""G""]",chr2_233263166_A_G,5.2000e+01,None,"{""AF"":[0.051622],""AQ"":[52],""AC"":null,""AN"":null}",chr2_233263166_A_G,5.6000e+01,None,"{""AF"":[0.05488],""AQ"":[56],""AC"":null,""AN"":null}",both


In [46]:
merged_pd.to_csv("/home/ubuntu/data/scripts/merged.tsv", sep="\t",index=False)

In [47]:
merged_pd_alleles=pd.merge(ukb_hits_pd, ibd_hits_pd, how='outer', on=['locus'], indicator="In both or unique", suffixes=('_ukbb', '_ibd' ))

In [48]:
merged_pd_alleles.to_csv("/home/ubuntu/data/scripts/merged_alleles.tsv", sep="\t",index=False)